In [27]:
import sys
sys.path.append("../_build/")

import constants
import pandas as pd
import numpy as np
import json
from datetime import datetime
from datetime import timedelta

In [65]:
def miles_hiked_per_day(df):
    completed = constants.get_completed(df)[[constants.DATE_COL, constants.TO_SPRINGER_COL]]
    completed[constants.DATE_COL+"_dt"] = completed[constants.DATE_COL].dt.strftime('%Y-%m-%d')

    # Need to shift the values, since the last checkpoint of each day should be the first checkpoint of the next day
    completed[constants.TO_SPRINGER_COL + '_shifted'] = completed[constants.TO_SPRINGER_COL].shift(1)
    completed[constants.DATE_COL + '_shifted'] = completed[constants.DATE_COL].shift(1)
    completed[constants.DATE_COL + '_dt_shifted'] = completed[constants.DATE_COL + '_dt'].shift(1)

    # Exclude calculating the overnight duration by filtering where the dates are different
    completed = completed[completed[constants.DATE_COL + '_dt'] == completed[constants.DATE_COL + '_dt_shifted']]

    # Group by day
    f = {constants.TO_SPRINGER_COL: 'last', 'to_spgr_shifted': 'first', constants.DATE_COL: 'last', constants.DATE_COL + '_shifted': 'first'}
    miles_per_day = completed.groupby(constants.DATE_COL+'_dt').agg(f)
    miles_per_day['miles'] = miles_per_day[constants.TO_SPRINGER_COL] - miles_per_day[constants.TO_SPRINGER_COL + '_shifted']
    miles_per_day['duration'] = miles_per_day[constants.DATE_COL] - miles_per_day[constants.DATE_COL + '_shifted']

    # Calculate Average Miles per Day (does not include zero days)
    avg_mileage = "0.0"
    if len(completed) > 0:
        avg_mileage = "{:.1f}".format(miles_per_day['miles'].mean())
    
    # Fill missing dates with zero miles
    miles_per_day.index = pd.DatetimeIndex(miles_per_day.index)
    start = miles_per_day.index[0]
    end = miles_per_day.index[-1]
    idx = pd.date_range(start, end)
    miles_per_day = miles_per_day.reindex(idx, fill_value=None)
    miles_per_day['miles'] = miles_per_day['miles'].fillna(0.0)
    miles_per_day['duration'] = miles_per_day['duration'].fillna(0.0)
    miles_per_day['to_spgr'] = miles_per_day['to_spgr'].fillna(method='ffill')
    
    # Calculate rolling 2 week mean over miles per day
    mean_excluding_zero = lambda x: x[np.nonzero(x)].mean()
    miles_per_day['rolling'] = miles_per_day['miles'].rolling(14, min_periods=1).apply(mean_excluding_zero)

    # Calculate how many miles I need to average to finish on time.  This assumes I take a zero every 10 days (* 0.9)
    estimated_finish = datetime.strptime(constants.ESTIMATED_FINISH_DT , '%Y-%m-%d')
    
    miles_per_day['days_to_finish'] = (estimated_finish - miles_per_day.index).total_seconds() * 0.9 / (24 * 60 * 60)
    miles_per_day['miles_remaining'] = df[constants.TO_SPRINGER_COL].iloc[-1] - miles_per_day['to_spgr']
    miles_per_day['need_to_average'] = miles_per_day['miles_remaining'] / miles_per_day['days_to_finish']
    
    # Fix rounding issue by converting each mileage value into a string
    miles_per_day['date'] = miles_per_day.index
    miles_per_day['date'] = miles_per_day['date'].dt.strftime('%Y-%m-%d')
    miles_per_day['miles'] = miles_per_day['miles'].apply(lambda x: float('{:.1f}'.format(x)))
    miles_per_day['duration'] = miles_per_day['duration'].apply(lambda x: int('{:.0f}'.format(x.total_seconds() / 60)))
    miles_per_day['rolling'] = miles_per_day['rolling'].apply(lambda x: float('{:.1f}'.format(x)))
    miles_per_day['need_to_average'] = miles_per_day['need_to_average'].apply(lambda x: float('{:.1f}'.format(x)))

    # Fill in the missing days so we can calculate the number of zero days
    num_zeros = 0
    if len(completed) > 0:
        zeros = miles_per_day.copy()
        idx = pd.date_range(zeros.index.min(), zeros.index.max())
        zeros.index = pd.DatetimeIndex(zeros.index)
        zeros = zeros.reindex(idx, fill_value=0)
        num_zeros = len(zeros[zeros['miles'] == 0])

    return {'mileage': miles_per_day[['date', 'miles', 'duration', 'rolling', 'need_to_average']].to_dict('records'), 'avg_mileage': avg_mileage, 'num_zeros': num_zeros}

In [66]:
checkpoints = constants.read_poi_file()
mpd = miles_hiked_per_day(checkpoints)

print(mpd)

{'mileage': [{'date': '2017-03-13', 'miles': 7.3, 'duration': 214, 'rolling': 7.3, 'need_to_average': 14.9}, {'date': '2017-03-14', 'miles': 5.8, 'duration': 205, 'rolling': 6.6, 'need_to_average': 15.0}, {'date': '2017-03-15', 'miles': 3.8, 'duration': 104, 'rolling': 5.6, 'need_to_average': 15.1}, {'date': '2017-03-16', 'miles': 7.7, 'duration': 302, 'rolling': 6.2, 'need_to_average': 15.1}, {'date': '2017-03-17', 'miles': 8.4, 'duration': 325, 'rolling': 6.6, 'need_to_average': 15.1}, {'date': '2017-03-18', 'miles': 7.2, 'duration': 259, 'rolling': 6.7, 'need_to_average': 15.2}, {'date': '2017-03-19', 'miles': 11.5, 'duration': 325, 'rolling': 7.4, 'need_to_average': 15.2}, {'date': '2017-03-20', 'miles': 9.7, 'duration': 240, 'rolling': 7.7, 'need_to_average': 15.2}, {'date': '2017-03-21', 'miles': 0.0, 'duration': 0, 'rolling': 7.7, 'need_to_average': 15.3}, {'date': '2017-03-22', 'miles': 10.4, 'duration': 296, 'rolling': 8.0, 'need_to_average': 15.3}, {'date': '2017-03-23', 'mil

In [68]:
df = constants.read_poi_file()    

completed = constants.get_completed(df)[[constants.DATE_COL, constants.TO_SPRINGER_COL]]
completed[constants.DATE_COL+"_dt"] = completed[constants.DATE_COL].dt.strftime('%Y-%m-%d')

print(len(completed))
print(len(df))
completed.head(10)

439
745


,dt_reached,to_spgr,dt_reached_dt
0,2017-03-13 10:03:00,-8.8,2017-03-13
1,2017-03-13 13:37:00,-1.5,2017-03-13
2,2017-03-14 11:37:00,-1.5,2017-03-14
3,2017-03-14 12:14:00,0.0,2017-03-14
4,2017-03-14 12:25:00,0.2,2017-03-14
5,2017-03-14 12:47:00,1.0,2017-03-14
6,2017-03-14 13:18:00,2.0,2017-03-14
7,2017-03-14 13:42:00,2.8,2017-03-14
8,2017-03-14 15:02:00,4.3,2017-03-14
9,2017-03-15 12:02:00,4.3,2017-03-15


In [69]:
# Need to shift the values, since the last checkpoint of each day should be the first checkpoint of the next day
completed[constants.TO_SPRINGER_COL + '_shifted'] = completed[constants.TO_SPRINGER_COL].shift(1)
completed[constants.DATE_COL + '_shifted'] = completed[constants.DATE_COL].shift(1)
completed[constants.DATE_COL + '_dt_shifted'] = completed[constants.DATE_COL + '_dt'].shift(1)

completed.head(10)

,dt_reached,to_spgr,dt_reached_dt,to_spgr_shifted,dt_reached_shifted,dt_reached_dt_shifted
0,2017-03-13 10:03:00,-8.8,2017-03-13,NaN,NaT,NaN
1,2017-03-13 13:37:00,-1.5,2017-03-13,-8.8,2017-03-13 10:03:00,2017-03-13
2,2017-03-14 11:37:00,-1.5,2017-03-14,-1.5,2017-03-13 13:37:00,2017-03-13
3,2017-03-14 12:14:00,0.0,2017-03-14,-1.5,2017-03-14 11:37:00,2017-03-14
4,2017-03-14 12:25:00,0.2,2017-03-14,0.0,2017-03-14 12:14:00,2017-03-14
5,2017-03-14 12:47:00,1.0,2017-03-14,0.2,2017-03-14 12:25:00,2017-03-14
6,2017-03-14 13:18:00,2.0,2017-03-14,1.0,2017-03-14 12:47:00,2017-03-14
7,2017-03-14 13:42:00,2.8,2017-03-14,2.0,2017-03-14 13:18:00,2017-03-14
8,2017-03-14 15:02:00,4.3,2017-03-14,2.8,2017-03-14 13:42:00,2017-03-14
9,2017-03-15 12:02:00,4.3,2017-03-15,4.3,2017-03-14 15:02:00,2017-03-14


In [70]:
# Group by day
f = {constants.TO_SPRINGER_COL: 'last', 'to_spgr_shifted': 'first', constants.DATE_COL: 'last', constants.DATE_COL + '_shifted': 'first'}
miles_per_day = completed.groupby(constants.DATE_COL+'_dt').agg(f)
miles_per_day['miles'] = miles_per_day[constants.TO_SPRINGER_COL] - miles_per_day[constants.TO_SPRINGER_COL + '_shifted']
miles_per_day['duration'] = miles_per_day[constants.DATE_COL] - miles_per_day[constants.DATE_COL + '_shifted']

# Fill missing dates with zero miles
miles_per_day.index = pd.DatetimeIndex(miles_per_day.index)
start = miles_per_day.index[0]
end = miles_per_day.index[-1]
idx = pd.date_range(start, end)
miles_per_day = miles_per_day.reindex(idx, fill_value=None)
miles_per_day['miles'] = miles_per_day['miles'].fillna(0.0)
miles_per_day['to_spgr'] = miles_per_day['to_spgr'].fillna(method='ffill')

miles_per_day.head(10)

,to_spgr,to_spgr_shifted,dt_reached,dt_reached_shifted,miles,duration
2017-03-13,-1.5,-8.8,2017-03-13 13:37:00,2017-03-13 10:03:00,7.3,0 days 03:34:00
2017-03-14,4.3,-1.5,2017-03-14 15:02:00,2017-03-13 13:37:00,5.8,1 days 01:25:00
2017-03-15,8.1,4.3,2017-03-15 13:46:00,2017-03-14 15:02:00,3.8,0 days 22:44:00
2017-03-16,15.8,8.1,2017-03-16 13:58:00,2017-03-15 13:46:00,7.7,1 days 00:12:00
2017-03-17,24.2,15.8,2017-03-17 14:34:00,2017-03-16 13:58:00,8.4,1 days 00:36:00
2017-03-18,31.4,24.2,2017-03-18 11:35:00,2017-03-17 14:34:00,7.2,0 days 21:01:00
2017-03-19,42.9,31.4,2017-03-19 12:54:00,2017-03-18 11:35:00,11.5,1 days 01:19:00
2017-03-20,52.6,42.9,2017-03-20 11:58:00,2017-03-19 12:54:00,9.7,0 days 23:04:00
2017-03-21,52.6,NaN,NaT,NaT,0.0,NaT
2017-03-22,63.0,52.6,2017-03-22 14:02:00,2017-03-20 11:58:00,10.4,2 days 02:04:00


In [72]:
estimated_finish = datetime.strptime(constants.ESTIMATED_FINISH_DT , '%Y-%m-%d')

miles_per_day['days_to_finish'] = (estimated_finish - miles_per_day.index).total_seconds() * 0.9 / (24 * 60 * 60) 
miles_per_day['miles_remaining'] = df[constants.TO_SPRINGER_COL].iloc[-1] - miles_per_day['to_spgr']
miles_per_day['need_to_average'] = miles_per_day['miles_remaining'] / miles_per_day['days_to_finish']

miles_per_day.head(1000)

,to_spgr,to_spgr_shifted,dt_reached,dt_reached_shifted,miles,duration,days_to_finish,miles_remaining,need_to_average
2017-03-13,-1.5,-8.8,2017-03-13 13:37:00,2017-03-13 10:03:00,7.3,0 days 03:34:00,146.7,2191.3,14.937287
2017-03-14,4.3,-1.5,2017-03-14 15:02:00,2017-03-13 13:37:00,5.8,1 days 01:25:00,145.8,2185.5,14.989712
2017-03-15,8.1,4.3,2017-03-15 13:46:00,2017-03-14 15:02:00,3.8,0 days 22:44:00,144.9,2181.7,15.056591
2017-03-16,15.8,8.1,2017-03-16 13:58:00,2017-03-15 13:46:00,7.7,1 days 00:12:00,144.0,2174.0,15.097222
2017-03-17,24.2,15.8,2017-03-17 14:34:00,2017-03-16 13:58:00,8.4,1 days 00:36:00,143.1,2165.6,15.133473
2017-03-18,31.4,24.2,2017-03-18 11:35:00,2017-03-17 14:34:00,7.2,0 days 21:01:00,142.2,2158.4,15.178622
2017-03-19,42.9,31.4,2017-03-19 12:54:00,2017-03-18 11:35:00,11.5,1 days 01:19:00,141.3,2146.9,15.193914
2017-03-20,52.6,42.9,2017-03-20 11:58:00,2017-03-19 12:54:00,9.7,0 days 23:04:00,140.4,2137.2,15.222222
2017-03-21,52.6,NaN,NaT,NaT,0.0,NaT,139.5,2137.2,15.320430
2017-03-22,63.0,52.6,2017-03-22 14:02:00,2017-03-20 11:58:00,10.4,2 days 02:04:00,138.6,2126.8,15.344877


In [24]:
# Fix rounding issue by converting each mileage value into a string
miles_per_day['date'] = miles_per_day.index
miles_per_day['date'] = miles_per_day['date'].dt.strftime('%Y-%m-%d')
miles_per_day['miles'] = miles_per_day['miles'].apply(lambda x: float('{:.1f}'.format(x)))
miles_per_day['duration'] = miles_per_day['duration'].apply(lambda x: int('{:.0f}'.format(x.total_seconds() / 60)))

mpd = {'mileage': miles_per_day[['date', 'miles', 'duration']].to_dict('records')}

print(mpd)

{'mileage': [{'date': '2017-03-13', 'miles': 7.3, 'duration': 214}, {'date': '2017-03-14', 'miles': 5.8, 'duration': 1525}, {'date': '2017-03-15', 'miles': 3.8, 'duration': 1364}, {'date': '2017-03-16', 'miles': 7.7, 'duration': 1452}, {'date': '2017-03-17', 'miles': 8.4, 'duration': 1476}, {'date': '2017-03-18', 'miles': 7.2, 'duration': 1261}, {'date': '2017-03-19', 'miles': 11.5, 'duration': 1519}, {'date': '2017-03-20', 'miles': 9.7, 'duration': 1384}, {'date': '2017-03-21', 'miles': 0.0, 'duration': 0}, {'date': '2017-03-22', 'miles': 10.4, 'duration': 3004}, {'date': '2017-03-23', 'miles': 10.8, 'duration': 1473}, {'date': '2017-03-24', 'miles': 12.2, 'duration': 1440}, {'date': '2017-03-25', 'miles': 16.3, 'duration': 1533}, {'date': '2017-03-26', 'miles': 7.2, 'duration': 1093}, {'date': '2017-03-27', 'miles': 0.0, 'duration': 0}, {'date': '2017-03-28', 'miles': 15.8, 'duration': 3295}, {'date': '2017-03-29', 'miles': 11.7, 'duration': 1139}, {'date': '2017-03-30', 'miles': 6.7

In [26]:
with open(constants.statsFilePath, 'w') as outfile:
    json.dump({**mpd}, outfile)